In [6]:
import pandas as pd
import numpy as np  

## I. Get matches from patch >= 55 (7.36),
Preprocess throw, loss, comeback, stomp empty values filled by 0

In [12]:
cols_to_read = [
    'match_id', 'duration', 'radiant_win', 
    'tower_status_radiant', 'tower_status_dire', 'barracks_status_radiant', 
    'barracks_status_dire', 'first_blood_time', 'radiant_score', 'dire_score',
    'radiant_team_id', 'dire_team_id', 'throw', 'loss', 'comeback', 'stomp', 'patch'
]
def read_metadata(csv_paths):
    # List to store DataFrames
    df_list = []
    
    for csv_path in csv_paths:
        # Read the CSV file
        df = pd.read_csv(csv_path, usecols=cols_to_read, index_col='match_id')
        
        # Filter rows based on 'patch' value
        df = df[df['patch'] >= 55]
        
        # Convert to decimal the status of barracks and tower status
        # Uncomment and modify the following lines if needed
        # df[['barracks_status_dire', 'barracks_status_radiant', 
        #     'tower_status_dire', 'tower_status_radiant']] = df[['barracks_status_dire','barracks_status_radiant' ,
        #                                                         'tower_status_dire','tower_status_radiant']].map(
        #     lambda x: int(str(x).strip().replace("'", ""), 2)
        # )
        
        # Fill NaN values with 0 in specified columns
        df[['throw', 'loss', 'comeback', 'stomp']] = df[['throw', 'loss', 'comeback', 'stomp']].fillna(0)
        
        # Drop the 'patch' column
        df = df.drop(columns=['patch'])
        
        # Append the DataFrame to the list
        df_list.append(df)
    
    # Concatenate all DataFrames in the list into a single DataFrame
    final_df = pd.concat(df_list)
    

    return final_df

In [13]:
patch_folders = ['202405', '202406', '202407', '202408']
patch_to_read = [f'./data/{patch}/main_metadata.csv/main_metadata.csv' for patch in patch_folders]

dota_df = read_metadata(patch_to_read)
dota_df

,barracks_status_dire,barracks_status_radiant,dire_score,duration,first_blood_time,radiant_score,radiant_win,tower_status_dire,tower_status_radiant,throw,loss,comeback,stomp,dire_team_id,radiant_team_id
match_id,,,,,,,,,,,,,,,
7750912161,'00111100','00110011',52,2075,148,36,False,'0000011110110000','0000000100000110',0.0,0.0,1210.0,10223.0,8629317.0,9425660.0
7750914469,'00111111','00110000',23,1856,206,16,False,'0000011100100100','0000011100000000',0.0,0.0,9441.0,1093.0,8629318.0,8629005.0
7750915644,'00111111','00001111',37,1850,251,26,False,'0000011110100110','0000011000111111',0.0,0.0,3335.0,8359.0,9330489.0,8961813.0
7750937564,'00111111','00000011',44,2525,3,35,False,'0000011110000100','0000011000000110',0.0,0.0,801.0,20711.0,9395679.0,9344594.0
7750968496,'00111111','00110011',32,1513,117,27,False,'0000011111110110','0000011100000100',0.0,0.0,2447.0,12041.0,8961813.0,9330489.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7881636100,'00111111','00001111',33,1379,195,11,False,'0000011111110110','0000011000100110',0.0,0.0,10.0,20459.0,8957156.0,8970060.0
7881664207,'00110011','00001111',12,1793,154,26,True,'0000000111000100','0000011000111110',1746.0,9749.0,0.0,0.0,8970060.0,8957156.0
7881677439,'00111111','00110100',32,2944,192,28,False,'0000011100100100','0000001100000000',0.0,0.0,8328.0,23065.0,2163.0,9255039.0


## II. Generate sparse matrix for hero selection for winning teams

In [14]:
def read_draft(csv_paths, matches):
    all_picks_bans = []  # List to store DataFrames from each CSV file
    
    for csv_path in csv_paths:
        # Read each CSV file
        picks_bans = pd.read_csv(csv_path)
        
        # Filter by match_id and drop unnecessary columns
        picks_bans = picks_bans[picks_bans['match_id'].isin(matches.index)]
        picks_bans = picks_bans.drop([picks_bans.columns[0], 'ord', 'leagueid'], axis=1, errors='ignore').reset_index(drop=True)
        
        # Append to the list
        all_picks_bans.append(picks_bans)
    
    # Concatenate all DataFrames into a single DataFrame
    combined_picks_bans = pd.concat(all_picks_bans, ignore_index=True)
    
    return combined_picks_bans

In [15]:
picks_bans_to_read = [f'./data/{patch}/picks_bans.csv/picks_bans.csv' for patch in patch_folders]
picks_bans = read_draft(picks_bans_to_read, dota_df)
picks_bans

,is_pick,hero_id,team,order,match_id
0,False,78.0,1.0,0.0,7750912161
1,False,95.0,0.0,1.0,7750912161
2,False,51.0,0.0,2.0,7750912161
3,False,9.0,1.0,3.0,7750912161
4,False,63.0,0.0,4.0,7750912161
...,...,...,...,...,...
146108,NaN,94.0,1.0,19.0,7881723710
146109,NaN,20.0,1.0,20.0,7881723710
146110,NaN,70.0,0.0,21.0,7881723710
146111,NaN,7.0,0.0,22.0,7881723710


### Filter out winning drafts

In [18]:
winning_drafts = pd.DataFrame()

def retrieve_wins(matches, draft):
    
    result_df = pd.DataFrame()
    # Iterate through each row in matches
    for _, row in matches.iterrows():
        match_id = row.name  # Access match_id from the index
        radiant_win = row['radiant_win']
        
        # Determine which team won to use as filter
        team_filter = 0 if radiant_win else 1
        
        # Filter picks_bans for the current match_id and team
        filtered_picks_bans = draft[(draft.match_id == match_id) & (draft['team'] == team_filter)]
        
        # Append the filtered results to result_df
        result_df = pd.concat([result_df, filtered_picks_bans], ignore_index=True)
    return result_df

wins = retrieve_wins(dota_df, picks_bans)

# Future proofing code for multiple csvs
winning_drafts = pd.concat([winning_drafts, wins], ignore_index=True)
winning_drafts['order'] = winning_drafts['order'] + 1


In [19]:
winning_drafts

,is_pick,hero_id,team,order,match_id
0,False,78.0,1.0,1.0,7750912161
1,False,9.0,1.0,4.0,7750912161
2,False,77.0,1.0,7.0,7750912161
3,True,59.0,1.0,8.0,7750912161
4,False,66.0,1.0,10.0,7750912161
...,...,...,...,...,...
73055,NaN,69.0,0.0,15.0,7881723710
73056,NaN,49.0,0.0,18.0,7881723710
73057,NaN,95.0,0.0,19.0,7881723710
73058,NaN,70.0,0.0,22.0,7881723710


In [20]:

def generate_sparse_matrix(matches, winning_drafts):
    num_rows = matches.shape[0]
    num_cols = 138  # Define the number of columns
    
    # Create the DataFrame with values from 1 to 124
    sparse_draft = pd.DataFrame(
        np.zeros((num_rows, num_cols)),
        index=matches.index,
        columns=range(1, num_cols + 1)  # Set column names from 1 to 124
    )
    
    # Iterate through each row in winning_drafts
    for _, row in winning_drafts.iterrows():
        match_id = row['match_id']
        hero_id = row['hero_id']
        order = row['order']
        
        # Update the sparse_draft DataFrame
        if match_id in sparse_draft.index:
            sparse_draft.loc[match_id, hero_id] = order
    
    return sparse_draft

# Assuming dota_df and winning_drafts are your DataFrames
sparse_draft = generate_sparse_matrix(dota_df, winning_drafts)
sparse_draft


,1,2,3,4,5,6,7,8,9,10,...,129,130,131,132,133,134,135,136,137,138
match_id,,,,,,,,,,,,,,,,,,,,,
7750912161,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7750914469,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,18.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0,0.0,0.0
7750915644,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0
7750937564,24.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7750968496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7881636100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7881664207,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7881677439,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
sparse_draft.to_csv('./data/sparse_matrix.csv')